# Funciones de agregación

Las funciones de agregación realizan una operación específica sobre todas las filas en una tabla. Estas funciones usualmente devuelven un valor resumiendo los datos de las columnas que nosotros consultemos. Las funciones de agregación implementadas en cada motor de bases de datos pueden variar, pero las 5 más comunes (y que están implementadas en `SQLite`) son: 

- Min: Valor Mínimo
- Max:  Valor Máximo
- Avg: Media (Promedio)
- Sum: Suma total de los valores 
- Count: Cantidad de elementos **No Nulos**

La sintaxis para utilizar estas funciones corresponde a la siguiente: 

```SQL
SELECT FUNC(column) FROM Tabla -- Donde FUNC, puede ser una de las 5 declaradas funciones de agregación declaradas anteriormente
```

A modo de ejemplo utilizando la tabla `Track`, podemos ver los valores mínimos y máximos para la duración de las pistas musicales:

In [2]:
import sqlite3 #Importamos la libreria

#Generamos la conexion
conn = sqlite3.connect("db//Chinook_Sqlite.sqlite") #En este caso, corresponde al archivo Chinook_Sqlite
cursor = conn.cursor() #Generamos nuestro cursor para las consultas

In [3]:
#Obtenemos el valor minimo de los milisegundos
cursor.execute("SELECT MIN(Milliseconds) FROM Track")
print(cursor.fetchone()) #Si se fijan, dado que es una función de agregación, esta consulta solo deberia devolver 1 solo valor 

#Obtenermos el valor maximo
cursor.execute("SELECT MAX(Milliseconds) FROM Track")
print(cursor.fetchone())

#Y tal como otras consultas SQL, podemos consultar 1 o mas multiples valores
cursor.execute("SELECT MIN(Milliseconds), MAX(Milliseconds) FROM Track")
print(cursor.fetchone())

(1071,)
(5286953,)
(1071, 5286953)


In [4]:
cursor.execute("SELECT count(Total) FROM Invoice WHERE Total > 20")
print(cursor.fetchone())

(4,)


Adicionalmente, podemos hacer operaciones matemáticas sobre las funciones de agregación (de la misma forma que se podría hacer sobre una columna con valores numéricos). En este caso, transformaremos los milisegundos a minutos.

In [5]:
cursor.execute("SELECT COUNT(Milliseconds), AVG(Milliseconds) / (60*1000),  SUM(Milliseconds) / (60*1000) FROM Track")
names = [description[0] for description in cursor.description]
print(names)
print(cursor.fetchone())

['COUNT(Milliseconds)', 'AVG(Milliseconds) / (60*1000)', 'SUM(Milliseconds) / (60*1000)']
(3503, 6.559986868398515, 22979)


Finalmente, podemos agregar la cláusula WHERE para poder filtrar filas según criterios, y luego sobre estos registros filtrados, realizar la agregación:

In [6]:
cursor.execute("SELECT COUNT(Milliseconds), AVG(Milliseconds) / (60*1000),  SUM(Milliseconds) / (60*1000)  FROM Track WHERE GenreId = 1")
print(cursor.fetchall())

[(1297, 4.7318340529426886, 6137)]


**Nota**: Cabe destacar incluir columnas no agregadas en una consulta que exista por lo menos una función de agregación, puede que devuelva como respuesta un error (dependiendo del motor de base de datos, ej: mysql). En el caso de SQLite, debería devolver el valor de la columna la cual se asocia el valor agregado (para las funciones MIN y MAX), o el primer valor (para otras funciones de agregación).

# Creaciones de grupos: GROUP BY, HAVING

Anteriormente, cuando nosotros aplicábamos funciones de agregación sobre nuestras columnas/tablas, el motor de base de datos automáticamente genera un grupo (el cual indica que es toda la tabla). Esto se le conoce como los grupos **implícitos**, debido a que no existe una cláusula para detallar como generar estos grupos. Por otra parte, los grupos **Explícitos** se generan cuando nosotros damos la instrucción al motor de base de datos sobre como generar los grupos. Para esto, nosotros podemos utilizar la cláusula `GROUP BY` indicando cual columna nosotros queremos generar los grupos. La sintaxis para utilizar generar los grupos corresponde a la siguiente:

```SQL
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla GROUP BY column1
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla WHERE (condicion_logica) GROUP BY column1
```

Explorando con nuestros ejemplos: veamos qué pasa si hacemos los grupos, pero sin aplicar ninguna función de agregación:


In [7]:
cursor.execute("SELECT GenreId FROM Track GROUP BY GenreId")
values = cursor.fetchmany(5) #Obtenemos los primeros 5 elementos

for value in values:
    print(value)

(1,)
(2,)
(3,)
(4,)
(5,)


En este caso, se imprimirán los primeros 5 grupos (o valores únicos) asociados a los géneros musicales de las canciones presentes en la tabla Track. Apliquemos funciones de agregación ahora y veamos qué es lo que nos retorna nuestras consultas:

In [8]:
cursor.execute("SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) FROM Track GROUP BY GenreId")
values = cursor.fetchmany(5)

for value in values:
    print(value)

(1, 1297, 4.7318340529426886)
(2, 130, 4.862589615384615)
(3, 374, 5.162490730837789)
(4, 332, 3.9058974899598393)
(5, 12, 2.2440583333333333)


En este caso, para cada uno de los Géneros Musicales se tienen el conteo de valores no nulos y el promedio de minutos para cada uno de estos Géneros. Ahora aplicando un filtro con la cláusula WHERE:

In [9]:
cursor.execute("SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) FROM Track WHERE UnitPrice > 1.5 GROUP BY GenreId")
values = cursor.fetchall()

for value in values:
    print(value)

(18, 13, 43.759151282051285)
(19, 93, 35.75068369175627)
(20, 26, 48.52971730769231)
(21, 64, 42.92139635416667)
(22, 17, 26.421061764705883)


Para canciones con el precio unitario mayor a  1.5 , los grupos (y por ende también) los valores de conteo de elementos no nulos y promedios de minutos van a cambiar. En este caso, los identificadores de Género Musical parten desde el 18 a diferencia de la consulta sin filtrar.

Cabe destacar que el filtro de WHERE se realiza previamente a realizar los grupos. Por lo mismo, existe la forma de poder filtrar después de realizar los grupos utilizando la cláusula HAVING y cuya sintaxis es: 
```SQL
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla GROUP BY column1 HAVING (condicion_logica1)
SELECT column1, FUNC(column2), FUNC(column3), ..., FUNC(columnN) FROM Tabla WHERE (condicion_logica1) GROUP BY column1 HAVING (condicion_logica2)
```

Viendo un ejemplo utilizando nuestras tablas:

In [10]:
cursor.execute("SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) FROM Track GROUP BY GenreId HAVING Milliseconds between 100000 AND 300000") 
values = cursor.fetchall()

for value in values:
    print(value)

(2, 130, 4.862589615384615)
(3, 374, 5.162490730837789)
(4, 332, 3.9058974899598393)
(5, 12, 2.2440583333333333)
(6, 81, 4.505996296296296)
(7, 579, 3.8809877086931492)
(8, 58, 4.119629310344828)
(9, 48, 3.817235069444444)
(10, 43, 4.072848062015503)
(12, 24, 3.1527368055555556)
(13, 28, 4.95754880952381)
(14, 61, 3.66778087431694)
(16, 28, 3.748730357142857)
(23, 40, 4.400975416666667)
(25, 1, 2.91355)


La gracia de utilizar having, es que ahora nuestras condiciones lógicas pueden incluir agregaciones dependientes de los grupos:

In [11]:
cursor.execute("SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) FROM Track GROUP BY GenreId HAVING COUNT() < 50 ") 
values = cursor.fetchall()

for value in values:
    print(value)

(5, 12, 2.2440583333333333)
(9, 48, 3.817235069444444)
(10, 43, 4.072848062015503)
(11, 15, 3.6598333333333333)
(12, 24, 3.1527368055555556)
(13, 28, 4.95754880952381)
(15, 30, 5.049763333333333)
(16, 28, 3.748730357142857)
(17, 35, 2.969604761904762)
(18, 13, 43.759151282051285)
(20, 26, 48.52971730769231)
(22, 17, 26.421061764705883)
(23, 40, 4.400975416666667)
(25, 1, 2.91355)


# Ordenar los valores retornados de una consulta

Para poder ordenar los valores retornados de una consulta desde SQL podemos usar la sentencia `ORDER BY`, la cual tiene que ir al final de la consulta SQL. La sintaxis corresponde a la siguiente:

```SQL
SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY column2
SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY column2 DESC

SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY 2
SELECT column1, column2, column3, ..., columnN FROM Tabla ORDER BY 2 DESC
```

En este caso, podemos ver dos cosas interesantes, ya sea podemos ordenar por el nombre de la columna obtenida en la respuesta, o en su defecto según su índice (column2, en este caso corresponde al índice 2) partiendo desde el valor 1. Lo segundo que podemos observar, corresponde a incluir o no la cláusula `DESC`. Por defecto, `ORDER BY` ordena de forma **ascendente**, y utilizando `DESC` obtenemos la forma **descendente**. Veamos unos ejemplos extendiendo lo visto con `GROUP BY` y `HAVING`


In [12]:
cursor.execute("SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) FROM Track GROUP BY GenreId HAVING COUNT(*) < 50 ORDER BY 3") 
values = cursor.fetchall()

for value in values:
    print(value)

(5, 12, 2.2440583333333333)
(25, 1, 2.91355)
(17, 35, 2.969604761904762)
(12, 24, 3.1527368055555556)
(11, 15, 3.6598333333333333)
(16, 28, 3.748730357142857)
(9, 48, 3.817235069444444)
(10, 43, 4.072848062015503)
(23, 40, 4.400975416666667)
(13, 28, 4.95754880952381)
(15, 30, 5.049763333333333)
(22, 17, 26.421061764705883)
(18, 13, 43.759151282051285)
(20, 26, 48.52971730769231)


In [13]:
cursor.execute("SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) as prom FROM Track GROUP BY GenreId HAVING COUNT(*) < 50 ORDER BY prom DESC") 
values = cursor.fetchall()

for value in values:
    print(value)

(20, 26, 48.52971730769231)
(18, 13, 43.759151282051285)
(22, 17, 26.421061764705883)
(15, 30, 5.049763333333333)
(13, 28, 4.95754880952381)
(23, 40, 4.400975416666667)
(10, 43, 4.072848062015503)
(9, 48, 3.817235069444444)
(16, 28, 3.748730357142857)
(11, 15, 3.6598333333333333)
(12, 24, 3.1527368055555556)
(17, 35, 2.969604761904762)
(25, 1, 2.91355)
(5, 12, 2.2440583333333333)


In [14]:
cursor.execute("SELECT GenreId, COUNT(GenreId), AVG(Milliseconds)/(60*1000) as prom FROM Track GROUP BY GenreId HAVING COUNT() > 40 ORDER BY 2 ") 
values = cursor.fetchall()

for value in values:
    print(value)

(10, 43, 4.072848062015503)
(9, 48, 3.817235069444444)
(8, 58, 4.119629310344828)
(14, 61, 3.66778087431694)
(21, 64, 42.92139635416667)
(24, 74, 4.897792792792793)
(6, 81, 4.505996296296296)
(19, 93, 35.75068369175627)
(2, 130, 4.862589615384615)
(4, 332, 3.9058974899598393)
(3, 374, 5.162490730837789)
(7, 579, 3.8809877086931492)
(1, 1297, 4.7318340529426886)
